In [ ]:
# pip install ucimlrepo
# from ucimlrepo import fetch_ucirepo
# dataset = fetch_ucirepo(id=352)

In [ ]:
pip install -qqq gdown ydata-profiling

In [ ]:
import gdown

import numpy as np
import pandas as pd

import logging
from ydata_profiling import ProfileReport

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("pipeline.log", mode="w"), logging.StreamHandler()]
)

In [ ]:
gdown.download(id="1NicvRmwc45R4olrDRLUhsOAUfRjwampK", output="online_retail.xlsx", quiet=False)

In [ ]:
logger = logging.getLogger()

In [ ]:
path = '/content/online_retail.xlsx'

In [ ]:
try:
    logger.info(f"Reading input Excel file: {path}")
    df = pd.read_excel(path, engine="openpyxl")
    logger.info(f"Loaded data with shape {df.shape}")
except Exception as e:
    logger.error(f"Failed to read Excel file: {e}")
    raise

In [ ]:
logger.info("Profiling dataset and assessing data quality issues...")


In [ ]:
# Recording basic info about columns
column_types = df.dtypes.astype(str).to_dict()
logger.info(f"Column data types: {column_types}")
missing_counts = df.isnull().sum().to_dict()
logger.info(f"Missing values per column: {missing_counts}")

In [ ]:
# Identifying business logic anomalies
neg_quantity_count = (df["Quantity"] < 0).sum()
logger.info(f"Records with negative Quantity (potential returns): {neg_quantity_count}")
zero_price_count = (df["UnitPrice"] == 0).sum()
logger.info(f"Records with zero UnitPrice: {zero_price_count}")
# (In this dataset, negative quantities indicate returns/cancellations, and UnitPrice of 0 might indicate freebies or errors)

In [ ]:
# Data cleaning steps
logger.info("Cleaning data: removing records with missing CustomerID, non-positive quantities or prices, and other anomalies.")
# Drop records with missing CustomerID (cannot analyze without customer)
df_clean = df.dropna(subset=["CustomerID"]).copy()
# Remove returns or cancellations: keep only records with Quantity > 0
df_clean = df_clean[df_clean["Quantity"] > 0]
# Remove records with zero or negative price if any
df_clean = df_clean[df_clean["UnitPrice"] > 0]
# Convert data types
df_clean["CustomerID"] = df_clean["CustomerID"].astype(int)  # CustomerID was float (due to NaNs) -> int
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])  # ensure datetime type for dates


In [ ]:
# Log results of cleaning
logger.info(f"Data cleaned. New shape: {df_clean.shape}")
removed_count = len(df) - len(df_clean)
logger.info(f"Total records removed during cleaning: {removed_count}")

In [ ]:
PROFILE_REPORT = "/content/sample_data/online_retail_profile.html"


In [ ]:
# Generate a detailed profiling report (HTML)
try:
    profile = ProfileReport(df_clean, title="Online Retail Data Profiling Report", explorative=True)
    profile.to_file(PROFILE_REPORT)
    logger.info(f"Profiling report generated at {PROFILE_REPORT}")
except Exception as e:
    logger.warning(f"Could not generate profiling report: {e}")

In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean.info()

In [ ]:
removed_count